<a href="https://colab.research.google.com/github/SupunGurusinghe/ML-with-superstore-dataset/blob/main/Superstore.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# install pyspark
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Import SparkSession
from pyspark.sql import SparkSession
# Create a Spark Session
spark = SparkSession.builder.master("local[*]").getOrCreate()
# Check Spark Session Information
spark

# **Data Preprocessing**

## Importing packages

In [ ]:
import pandas as pd
import numpy as np

## Importing superstore dataset

In [74]:
url = "https://raw.githubusercontent.com/SupunGurusinghe/ML-with-superstore-dataset/main/superstore.csv"
superstore_df = pd.read_csv(url ,encoding='windows-1252')
superstore_df

,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,Country,City,...,Postal Code,Region,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit
0,1,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-BO-10001798,NaN,Bookcases,Bush Somerset Collection Bookcase,261.9600,2,0.00,41.9136
1,2,CA-2016-152156,11/8/2016,11/11/2016,Second Class,CG-12520,Claire Gute,Consumer,United States,Henderson,...,42420,South,FUR-CH-10000454,Furniture,Chairs,"Hon Deluxe Fabric Upholstered Stacking Chairs,...",731.9400,3,0.00,219.5820
2,3,CA-2016-138688,6/12/2016,6/16/2016,Second Class,DV-13045,Darrin Van Huff,Corporate,United States,Los Angeles,...,90036,West,OFF-LA-10000240,Office Supplies,Labels,Self-Adhesive Address Labels for Typewriters b...,14.6200,2,0.00,6.8714
3,4,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,FUR-TA-10000577,Furniture,Tables,Bretford CR4500 Series Slim Rectangular Table,957.5775,5,0.45,-383.0310
4,5,US-2015-108966,10/11/2015,10/18/2015,Standard Class,SO-20335,Sean O'Donnell,Consumer,United States,Fort Lauderdale,...,33311,South,OFF-ST-10000760,Office Supplies,Storage,Eldon Fold 'N Roll Cart System,22.3680,2,0.20,2.5164
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9989,9990,CA-2014-110422,1/21/2014,1/23/2014,Second Class,TB-21400,Tom Boeckenhauer,Consumer,United States,Miami,...,33180,South,FUR-FU-10001889,Furniture,Furnishings,Ultra Door Pull Handle,25.2480,3,0.20,4.1028
9990,9991,CA-2017-121258,2/26/2017,3/3/2017,Standard Class,DB-13060,Dave Brooks,Consumer,United States,Costa Mesa,...,92627,West,FUR-FU-10000747,Furniture,Furnishings,Tenex B1-RE Series Chair Mats for Low Pile Car...,91.9600,2,0.00,15.6332
9991,9992,CA-2017-121258,2/26/2017,3/3/2017,Standard Class,DB-13060,Dave Brooks,Consumer,United States,Costa Mesa,...,92627,West,TEC-PH-10003645,Technology,Phones,Aastra 57i VoIP phone,258.5760,2,0.20,19.3932
9992,9993,CA-2017-121258,2/26/2017,3/3/2017,Standard Class,DB-13060,Dave Brooks,Consumer,United States,Costa Mesa,...,92627,West,OFF-PA-10004041,Office Supplies,Paper,"It's Hot Message Books with Stickers, 2 3/4"" x 5""",29.6000,4,0.00,13.3200


In [75]:
from pyspark.sql.types import *

sales_schema = StructType([
    StructField("RowID", IntegerType(), False),
    StructField("OrderID", StringType(), False),
    StructField("OrderDate", StringType(), False, metadata={"desc": "Date of order placed"}),
    StructField("ShipDate", StringType(), False, metadata={"desc": "Date of order shipped"}),
    StructField("ShipMode", StringType(), True, metadata={"desc": "Mode of shipment"}),
    StructField("CustomerID", StringType(), False),
    StructField("CustomerName", StringType(), True),
    StructField("Segment", StringType(), True, metadata={"desc": "Type of the customer"}),
    StructField("Country", StringType(), True),
    StructField("City", StringType(), False),
    StructField("State", StringType(), True),
    StructField("PostalCode", StringType(), True),
    StructField("Region", StringType(), True),
    StructField("ProductID", StringType(), False),
    StructField("Category", StringType(), True),
    StructField("SubCategory", StringType(), True),
    StructField("ProductName", StringType(), True),
    StructField("Sales", FloatType(), False),
    StructField("Quantity", IntegerType(), False),
    StructField("Discount", FloatType(), False),
    StructField("Profit", FloatType(), False),
])

In [76]:
superstore_df = spark.createDataFrame(superstore_df, sales_schema)
superstore_df.show()

+-----+--------------+----------+----------+--------------+----------+------------------+-----------+-------------+---------------+--------------+----------+-------+---------------+---------------+-----------+--------------------+--------+--------+--------+--------+
|RowID|       OrderID| OrderDate|  ShipDate|      ShipMode|CustomerID|      CustomerName|    Segment|      Country|           City|         State|PostalCode| Region|      ProductID|       Category|SubCategory|         ProductName|   Sales|Quantity|Discount|  Profit|
+-----+--------------+----------+----------+--------------+----------+------------------+-----------+-------------+---------------+--------------+----------+-------+---------------+---------------+-----------+--------------------+--------+--------+--------+--------+
|    1|CA-2016-152156| 11/8/2016|11/11/2016|  Second Class|  CG-12520|       Claire Gute|   Consumer|United States|      Henderson|      Kentucky|     42420|  South|FUR-BO-10001798|            NaN|  

## Handling null values

In [77]:
from pyspark.sql import functions

df_1 = superstore_df.groupBy(superstore_df['CustomerID']).agg(functions.first(superstore_df['CustomerName'], ignorenulls=True))
df_2 = superstore_df.groupBy(superstore_df['ProductID']).agg(functions.first(superstore_df['Category'], ignorenulls=True))

df_1.show()
df_2.show()

+----------+--------------------+
|CustomerID| first(CustomerName)|
+----------+--------------------+
|  AA-10315|          Alex Avila|
|  AA-10375|        Allen Armold|
|  AA-10480|        Andrew Allen|
|  AA-10645|       Anna Andreadi|
|  AB-10015|       Aaron Bergman|
|  AB-10060|     Adam Bellavance|
|  AB-10105|       Adrian Barton|
|  AB-10150|         Aimee Bixby|
|  AB-10165|         Alan Barnes|
|  AB-10255|Alejandro Ballentine|
|  AB-10600|           Ann Blume|
|  AC-10420|       Alyssa Crouse|
|  AC-10450|             Amy Cox|
|  AC-10615|           Ann Chong|
|  AC-10660|          Anna Chung|
|  AD-10180|      Alan Dominguez|
|  AF-10870|        Art Ferguson|
|  AF-10885|          Art Foster|
|  AG-10270|     Alejandro Grove|
|  AG-10300| Aleksandra Gannaway|
+----------+--------------------+
only showing top 20 rows

+---------------+---------------+
|      ProductID|first(Category)|
+---------------+---------------+
|FUR-BO-10000112|      Furniture|
|FUR-BO-10000330|     